# **Downloading the dataset directly from Kaggle**

In [3]:
# Upload kaggle.json file
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omupadhyaya","key":"9865addc96b37687aafdbc01e4a448fb"}'}

In [5]:
# Create Kaggle directory
!mkdir ~/.kaggle

In [6]:
#Copy kaggle.json
!cp kaggle.json ~/.kaggle/

In [7]:
!ls -ltr ~/.kaggle

total 4
-rw-r--r-- 1 root root 67 Dec 23 07:10 kaggle.json


In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!ls -ltr ~/.kaggle

total 4
-rw------- 1 root root 67 Dec 23 07:10 kaggle.json


In [10]:
!kaggle datasets download -d 'asdasdasasdas/garbage-classification'

Dataset URL: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification
License(s): copyright-authors
  0% 0.00/82.0M [00:00<?, ?B/s]
100% 82.0M/82.0M [00:00<00:00, 1.57GB/s]


In [11]:
!ls -ltr

total 83964
-rw-r--r-- 1 root root 85969666 Oct 13  2019 garbage-classification.zip
drwxr-xr-x 1 root root     4096 Dec  9 14:42 sample_data
-rw-r--r-- 1 root root       67 Dec 23 07:10 kaggle.json


In [12]:
!unzip /content/garbage-classification.zip -d ../content/

Streaming output truncated to the last 5000 lines.
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard152.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard153.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard154.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard155.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard156.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard157.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard158.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard159.jpg  
  inflating: ../content/Garbage classification/Garbage classification/cardboard/cardboard16.jpg  
  inflating: ../content/Garbage classification/Garbage clas

# **File Handling**

In [13]:
with open('/content/one-indexed-files-notrash_train.txt') as f:
  train_data = []
  for line in f:
    line = line.strip()
    filename, label = line.rsplit(maxsplit=1) # maxsplit = 1 means split only once. This acts as a safeguard in case file names have a white space
    train_data.append((filename, int(label)))

In [14]:
with open('/content/one-indexed-files-notrash_test.txt') as f:
  test_data = []
  for line in f:
    line = line.strip()
    filename, label = line.rsplit(maxsplit=1) # maxsplit = 1 means split only once. This acts as a safeguard in case file names have a white space
    test_data.append((filename, int(label)))

In [15]:
with open('/content/one-indexed-files-notrash_val.txt') as f:
  val_data = []
  for line in f:
    line = line.strip()
    filename, label = line.rsplit(maxsplit=1) # maxsplit = 1 means split only once. This acts as a safeguard in case file names have a white space
    val_data.append((filename, int(label)))

In [16]:
def FilenameToPath(List_Filename):
  CSV_Data = []
  for data in List_Filename:
    filename = data[0]
    label = data[1]

    match(label):
      case 1:
        path = '/content/Garbage classification/Garbage classification/glass/' + filename
      case 2:
        path = '/content/Garbage classification/Garbage classification/paper/' + filename
      case 3:
        path = '/content/Garbage classification/Garbage classification/cardboard/' + filename
      case 4:
        path = '/content/Garbage classification/Garbage classification/plastic/' + filename
      case 5:
        path = '/content/Garbage classification/Garbage classification/metal/' + filename
      case 6:
        path = '/content/Garbage classification/Garbage classification/trash/' + filename
      case _:
        raise RuntimeError('INVALID LABEL FOUND!')

    CSV_Data.append((path, label))

  return CSV_Data

In [17]:
train_data_csv = FilenameToPath(train_data)
test_data_csv = FilenameToPath(test_data)
val_data_csv = FilenameToPath(val_data)

In [18]:
import csv

with open('Train_Data.csv', 'w', newline='') as f:
  writer = csv.writer(f)

  for data in train_data_csv:
    writer.writerow(data)


In [19]:
with open('Test_Data.csv', 'w', newline='') as f:
  writer = csv.writer(f)

  for data in test_data_csv:
    writer.writerow(data)


In [20]:
with open('Val_Data.csv', 'w', newline='') as f:
  writer = csv.writer(f)

  for data in val_data_csv:
    writer.writerow(data)


# **Data Preprocessing**

In [21]:
import pandas as pd
import numpy as np

In [22]:
train_df = pd.read_csv('/content/Train_Data.csv', names = ['Image_Path', 'Category'])
train_df

,Image_Path,Category
0,/content/Garbage classification/Garbage classi...,3
1,/content/Garbage classification/Garbage classi...,2
2,/content/Garbage classification/Garbage classi...,2
3,/content/Garbage classification/Garbage classi...,1
4,/content/Garbage classification/Garbage classi...,1
...,...,...
1763,/content/Garbage classification/Garbage classi...,3
1764,/content/Garbage classification/Garbage classi...,1
1765,/content/Garbage classification/Garbage classi...,5
1766,/content/Garbage classification/Garbage classi...,4


In [23]:
test_df = pd.read_csv('/content/Test_Data.csv', names = ['Image_Path', 'Category'])
test_df

,Image_Path,Category
0,/content/Garbage classification/Garbage classi...,2
1,/content/Garbage classification/Garbage classi...,2
2,/content/Garbage classification/Garbage classi...,3
3,/content/Garbage classification/Garbage classi...,1
4,/content/Garbage classification/Garbage classi...,2
...,...,...
426,/content/Garbage classification/Garbage classi...,5
427,/content/Garbage classification/Garbage classi...,2
428,/content/Garbage classification/Garbage classi...,2
429,/content/Garbage classification/Garbage classi...,2


In [24]:
val_df = pd.read_csv('/content/Val_Data.csv', names = ['Image_Path', 'Category'])
val_df

,Image_Path,Category
0,/content/Garbage classification/Garbage classi...,3
1,/content/Garbage classification/Garbage classi...,4
2,/content/Garbage classification/Garbage classi...,1
3,/content/Garbage classification/Garbage classi...,1
4,/content/Garbage classification/Garbage classi...,1
...,...,...
323,/content/Garbage classification/Garbage classi...,5
324,/content/Garbage classification/Garbage classi...,3
325,/content/Garbage classification/Garbage classi...,4
326,/content/Garbage classification/Garbage classi...,1


In [25]:
# Data Cleaning
train_df.dropna()
test_df.dropna()
val_df.dropna()

,Image_Path,Category
0,/content/Garbage classification/Garbage classi...,3
1,/content/Garbage classification/Garbage classi...,4
2,/content/Garbage classification/Garbage classi...,1
3,/content/Garbage classification/Garbage classi...,1
4,/content/Garbage classification/Garbage classi...,1
...,...,...
323,/content/Garbage classification/Garbage classi...,5
324,/content/Garbage classification/Garbage classi...,3
325,/content/Garbage classification/Garbage classi...,4
326,/content/Garbage classification/Garbage classi...,1


In [26]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
# cv2.imshow() is disabled in Colab, because it causes Jupyter sessions to crash

# **Analysing Images using OpenCV**

We analyse the images in training, validation and testing datasets using the OpenCV module. We inspect the datasets for the following three flags:

1. Do all images have the same dimensions or not? If not, then what is the minimum height and minimum width?
2. Are all the images square?
3. Do all images have 3 channels?

In [27]:
def inspectImages(df):
  c1 = 0
  c2 = 0
  c3 = 0
  height_old = 0
  width_old = 0
  heights = []
  widths = []

  for row, col in df.iterrows():
    path = df.iloc[row,0]
    img = cv.imread(path)
    height, width, channels = img.shape

    # To check if all images have same dimensions.
    if (height_old != height or width_old != width) and (height_old != 0 and width_old != 0):
      c3 += 1

    # To check if the images are square.
    if height != width:
      c1 += 1

    # To check if all images have 3 channels.
    if channels != 3:
      c2 += 1

    height_old = height
    width_old = width

    if c3 > 0:
      heights.append(height)
      widths.append(width)

  print(c1, '/', df.shape[0], 'images are rectangular.')

  if len(heights) > 0:
    print('Minimum height:', min(heights))
  else:
    print('All images have height', height_old)

  if len(widths) > 0:
    print('Minimum width:', min(widths))
  else:
    print('All images have width', width_old)

  print(c3, '/', df.shape[0], "images have different dimensions.")
  print('Channels not equal to 3 for', c2, '/', df.shape[0], 'images.')

In [28]:
print("Inspecting the training dataset...")
inspectImages(train_df)
print("\nInspecting the validation dataset...")
inspectImages(val_df)
print("\nInspecting the testing dataset...")
inspectImages(test_df)

Inspecting the training dataset...
1768 / 1768 images are rectangular.
All images have height 384
All images have width 512
0 / 1768 images have different dimensions.
Channels not equal to 3 for 0 / 1768 images.

Inspecting the validation dataset...
328 / 328 images are rectangular.
All images have height 384
All images have width 512
0 / 328 images have different dimensions.
Channels not equal to 3 for 0 / 328 images.

Inspecting the testing dataset...
431 / 431 images are rectangular.
All images have height 384
All images have width 512
0 / 431 images have different dimensions.
Channels not equal to 3 for 0 / 431 images.


**Conclusion:**
All images in the training, validation and testing datasets have 3 channels and dimensions 384 x 512. This means we do not need to resize the images unless we use the model for images from external sources.

# **Preparing the datasets for PyTorch operations:**

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
from torchvision.io import decode_image

In [30]:
class MyDataset(Dataset):
  def __init__(self, df, transform=None):
    self.df = df
    self.transform = transform

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    img_path, category = self.df.iloc[idx, 0], torch.tensor(self.df.iloc[idx, 1], dtype = torch.long)
    img = decode_image(img_path).to(torch.float32) / 255.0

    if self.transform != None:
      img = self.transform(img)

    return img, category

In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [32]:
train_data = MyDataset(train_df, transform = transform)
val_data = MyDataset(val_df, transform = transform)
test_data = MyDataset(test_df, transform = transform)

In [33]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers = 0, pin_memory=True)
val_loader = DataLoader(val_data, batch_size = 64, shuffle = True, num_workers = 0, pin_memory=True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = True, num_workers = 0, pin_memory=True)

In [34]:
image, label = train_data[0]
image.size()

torch.Size([3, 224, 224])

In [35]:
train_data.__len__()

1768

# **Defining the CNN Architecture**

Work in Progress

In [45]:
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 4) # (3, 256, 256) -> (32, 253, 253)
    self.pool = nn.MaxPool2d(2, 2)  # (32, 126, 126)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 96, kernel_size = 4)  # (32, 126, 126) -> (96, 123, 123) -> (96, 61, 61)
    self.gap = nn.AdaptiveAvgPool2d((1, 1))
    self.fc1 = nn.Linear(96, 256)

    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 6)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.gap(x)

    x = torch.flatten(x, start_dim = 1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)

    return x

In [46]:
net = NeuralNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 1e-3)

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [48]:
image, label = train_data[0]
image.shape

torch.Size([3, 224, 224])

In [49]:
scaler = torch.cuda.amp.GradScaler()

/tmp/ipython-input-2340218076.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [50]:
print(torch.cuda.get_device_name(0))

Tesla T4


In [52]:
net.train()

for epoch in range(15):
    print(f"Training epoch {epoch+1}")
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.to(device, non_blocking=True)
        labels = (labels - 1).to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            outputs = net(inputs)
            loss = loss_fn(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    print(f"Loss: {running_loss/len(train_loader):.4f}")


Training epoch 1


KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)  # Make sure model is on the right device

correct = 0
total = 0

net.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)  # Send images to same device as model
        labels = labels.to(device)  # Send labels to same device (optional for comparison)
        outputs = net(images)
        _, prediction = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (prediction == labels-1).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')
